In [1]:
import torch
from torch import nn, optim
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data.sampler import WeightedRandomSampler
# from torch.utils.data import WeightedRandomSampler

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import numpy as np
from sklearn.manifold import TSNE


## T-SNE & Make GIF FILE 

In [3]:
class Visualization:
    def __init__(self):
        self.labels = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]
    def TSNE(self, matrix,iter_num):
        model = TSNE(n_components=2, random_state=0)
        model.fit_transform(matrix)
        sns.scatterplot(matrix[:,0],matrix[:,1],hue = self.labels)
        plt.savefig('fig'+str(iter_num)+'.png', dpi=300)
        plt.close()

 ## load data 

In [4]:
class GraphData:
    def __init__(self, G):
        
        '''
        G: networkx 객체
        G.nodes, G.edges, G.neighbors(i) 
        '''
        self.G=G
        
    
    def NegativeSampler(self, v, k=2 ,with_replacement=False):
        '''
        k: sample의 개수
        i: vertex
        d_v^(3/4), where d_v = the oyt-degree of vertex v
        '''
        
        # v의 negative nodes set 구하기 
        neg_neighbor = set(self.G.nodes)-set(self.G.neighbors(v))-set([v])
        
        # out degree^(3/4) 구하기 
        out_degree = list( map ( lambda a: self.G.degree(a) **(3/4), neg_neighbor ) )
        
        return list(WeightedRandomSampler(out_degree, k, replacement=with_replacement))


## model

In [5]:
class LINE(nn.Module):
    def __init__(self, G, emd_dim, neg_k = 2, second_order = False):
        '''
        
        emd_dim: embedding dimension
        pos_k: # positive sample
        neg_k: # negative sample   
        
        '''
        super(LINE, self).__init__() 
        self.G = GraphData(G)
        self.V = len(G.nodes) # number of nodes
        self.neg_k = neg_k
        
        self.emd_dim = emd_dim
        
        self.emd_layer = nn.Embedding(num_embeddings = self.V, 
                               embedding_dim = emd_dim)
        
        self.second_order = second_order
        if self.second_order:
            self.context_layer = nn.Embedding(num_embeddings = self.V, 
                               embedding_dim = emd_dim)
     
        
        #self.log_sigmoid = F.logsigmoid(torch.sum(???, dim = 1))
        
   
    def forward(self, i, j, device):
        v_i = self.emd_layer(i).to(device)
        if self.second_order:
            v_j = self.context_layer(j).to(device)
        else:
            v_j = self.emd_layer(j).to(device)
            
        pos_loss = F.logsigmoid(torch.matmul(v_i, v_j)).to(device)
        
        neg_set = self.G.NegativeSampler(v = i.tolist() , k = self.neg_k)
        neg_set = torch.tensor(neg_set).to(device)
        neg_loss = 0
        
        for v_n in neg_set:
            if self.second_order:
                prob = torch.matmul(v_i,self.context_layer(v_n)).to(device)
                neg_loss += F.logsigmoid(prob).to(device)
                
            else: # first_order
                prob = torch.matmul(v_i,self.emd_layer(v_n)).to(device)
                neg_loss += F.logsigmoid(prob).to(device)
                
        negative_sampling = pos_loss + neg_loss
        
        return negative_sampling
        

## train

In [6]:
class TrainLINE:
    def __init__(self, G, emd_dim,epochs,learning_rate = 0.01, neg_k = 2, second_order = False):
        self.line = LINE(G = G,emd_dim = emd_dim, neg_k = 2, second_order = False)
        self.learning_rate = learning_rate
        self.batch = list(G.edges)
        self.nodes = len(G.nodes)
        self.epochs = epochs
    
    def train(self):
        opt = optim.SGD(self.line.parameters(), lr=self.learning_rate)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.line.cuda()
        
        #### for i in tqdm(range(1, 600)):
        for epoch in range(self.epochs):
            self.line.train()
            np.random.shuffle(self.batch)
            
            print("Epoch {}".format(epoch))
            
            for b in tqdm(self.batch):
                self.line.zero_grad()
                loss = self.line(i=torch.tensor(b[0],device=device), j=torch.tensor(b[1],device=device), device=device)
                loss.backward()
                opt.step()
                
                
            self.line.eval()
            Visualization().TSNE(self.line.emd_layer(torch.tensor(range(self.nodes),device=device)).cpu().detach().numpy(),epoch)
            

        print("\nDone training\n")        
        

In [7]:
model = TrainLINE(nx.karate_club_graph(), epochs = 100, learning_rate = 0.001, emd_dim = 5, neg_k=2, second_order = False )

In [8]:
model.train()

  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

Epoch 0


 50%|████████████████████████████████████████▌                                        | 39/78 [00:00<00:00, 383.38it/s]

Epoch 1


 51%|█████████████████████████████████████████▌                                       | 40/78 [00:00<00:00, 389.49it/s]

...

Epoch 98


 51%|█████████████████████████████████████████▌                                       | 40/78 [00:00<00:00, 393.09it/s]

Epoch 99


100%|█████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 401.07it/s]



Done training

